In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
epochs = 10
batch_size = 32
learning_rate = 1e-4

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Flatten, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder



def build_model(num_classes):
    base_model = InceptionResNetV2(include_top=False, input_shape=(300, 300, 3), weights='imagenet')

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = TimeDistributed(Dense(128))(x)
    x = TimeDistributed(Dropout(0.5))(x)
    x = TimeDistributed(Flatten())(x)

    lstm_1 = LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)
    lstm_2 = LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)
    x = Bidirectional(lstm_1)(x)
    x = Bidirectional(lstm_2)(x)

    x = TimeDistributed(Dense(64, activation='relu'))(x)
    x = TimeDistributed(Dropout(0.5))(x)
    x = TimeDistributed(Flatten())(x)

    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    return model



In [ ]:
model = build_model(num_classes=2)


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])





In [ ]:
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

In [ ]:
model.summary()

In [ ]:
model.save('drive/My Drive/ColabDrive/deepfake-detection-challenge/train_colab_videos/my_model_one.h5')